In [ ]:
import pandas as pd
import json
import gzip
import os

#### zcat ~/annDB/broad_hg38/Homo_sapiens.GRCh38.109.gff3.gz | grep "ID=gene" > gene.info

In [ ]:
file = "/home/marc/inters/output_test2/05_Annotation/Nirvana/snvs/Annotation.json.gz"
gene_info = "/home/marc/Desktop/data/refs/gtf/GRCH38_91/gene.info"
Jasix = os.path.expanduser("$HOME/Nirvana/bin/Release/net6.0/Jasix.dll")



In [ ]:
def extract_gene_region(Ann_file, gene_info, id, padding=1000):
    interval = os.popen(f"grep -i 'Name={id};' {gene_info} | awk -F'\t' '{{print \"chr\"$1\":\"$4-{padding}\"-\"$5+{padding}}}'").read().strip()
    
    print(F"Exporting {id} with padding [ {padding} ] ({interval})  from Annotation file...")
    dotnet_command = f"dotnet {Jasix} -i {Ann_file} -q {interval}"
    dotnet_output = os.popen(dotnet_command).read().replace("\n","")

    # Format dotnet output
    formatted_output = dotnet_output.replace("{  \"chromosome", "\n{\"chromosome")
    formatted_output = formatted_output.replace(" ", "")
    formatted_output = formatted_output.replace('{"positions":[\n{', '[{').replace("}]}]}]}","}]}]}]")
    # formatted_output = formatted_output.replace("]}]}", "}]\n").replace('{"positions":[\n',"[")

    return pd.DataFrame(json.loads(formatted_output))
    

    

In [ ]:
extract_gene_region(file, gene_info, "FBXO16", 5000)

In [ ]:
def process_region(*args):
    counter = 0
    for df in args:
        print(len(df))
        if counter == 0:
            my_df = df
        else:
            my_df = pd.concat([my_df, df], axis=0, ignore_index=True)
        counter += 1
        
    return my_df

In [ ]:
len(process_region(extract_gene_region(file, gene_info, "OR4F21", 5000),extract_gene_region(file, gene_info, "FBXO16", 5000),extract_gene_region(file, gene_info, "FGL1", 5000)))

In [ ]:
FBXO16

In [ ]:
egene = extract_gene_region(file, gene_info, "OR4F21", 5000)


In [ ]:
df = pd.DataFrame(egene)
variants_df = df['variants'].apply(pd.Series)
variants_df2 = variants_df[0].apply(pd.Series)
final_df = pd.concat([df, variants_df2], axis=1).drop(['variants', 'samples'], axis=1)

In [ ]:
print(f"Found {len(final_df)} variants in Gene (OR4F21).")


In [ ]:
final_df["dbsnp"].count()

In [ ]:
biotypes = set()
consequences = set()
for i in final_df['transcripts'].iloc[1]:
    biotypes.add( i["bioType"])
    
    consequences.add(j for j in i["consequence"])

In [ ]:
for i in myvar:
    print(i)

In [ ]:
print(j for j in myvar)

In [ ]:
final_df

In [ ]:
final_df.to_csv("/home/marc/OR4F21.tsv",sep='\t',index=False)

In [ ]:

header = ''
positions = []
genes = []
is_header_line = True
is_position_line = False
is_gene_line = False
gene_section_line = '],"genes":['
end_line = ']}'

with gzip.open(file, 'rt') as f:
    position_count = 0
    gene_count = 0
    for line in f:
        trim_line = line.strip()
        if is_header_line:
            ## only keep the "header" field content from the line
            header = trim_line[10:-14]
            is_header_line = False
            is_position_line = True
            continue
        if trim_line == gene_section_line:
            is_gene_line = True
            is_position_line = False
            continue
        elif trim_line == end_line:
            break
        else:
            if is_position_line:
                ## remove the trailing ',' if there is
                positions.append(trim_line.rstrip(','))
                position_count += 1
            if is_gene_line:
                ## remove the trailing ',' if there is
                genes.append(trim_line.rstrip(','))
                gene_count += 1

print ('number of positions:', position_count)
print ('number of genes:', gene_count)


In [ ]:
mylist = []
for gene in genes:
    position_dict = json.loads(gene)
    mylist.append(position_dict)
genedf = pd.DataFrame(mylist)

In [ ]:
mylist = []
for position in positions:
    print(position)
    position_dict = json.loads(position)
    mylist.append(position_dict)
df = pd.DataFrame(mylist)
variants_df = df['variants'].apply(pd.Series)
variants_df2 = variants_df[0].apply(pd.Series)
final_df = pd.concat([df, variants_df2], axis=1).drop(['variants', 'samples'], axis=1)



In [ ]:
final_df = pd.read_csv('/home/marc/inters/output_test2/05_Annotation/Nirvana/snvs/Annotation.tsv', delimiter='\t')


In [ ]:


# Define the gene symbol you want to search for
target_gene_symbol = 'OR4F21'
# final_df = pd.read_csv('/home/marc/inters/output_test2/05_Annotation/Nirvana/snvs/Annotation.tsv', delimiter='\t')

# Function to filter rows based on the target gene symbol
def filter_by_gene_symbol(row):
    if isinstance(row, list):
        for d in row:
            if isinstance(d, dict) and d.get('hgnc') == target_gene_symbol:
                return True
    return False

# Apply the filter function to the DataFrame
filtered_df = final_df[final_df['transcripts'].apply(filter_by_gene_symbol)]

# Print the filtered DataFrame
# print(filtered_df)

def is_novel(row):
    return pd.isna(row['dbsnp'])

total_rows = len(filtered_df)
total_variants = total_rows
variants_with_annotations = total_rows - filtered_df['dbsnp'].isna().sum()
novel_variants = total_variants - variants_with_annotations
percentage_novel = (novel_variants / total_rows) * 100
# Print the statistics
print()
print(f"#--------------------{target_gene_symbol}--------------------#")
print(f"Total Variants: {total_variants}")
print(f"Variants with Annotations: {variants_with_annotations}")
print(f"Variants without dbSNP Annotations: {novel_variants}")
print(f"Percentage of Novel Variants: {percentage_novel} %")

In [ ]:
import threading
import queue
import concurrent.futures

In [ ]:
header = ''
positions = []
genes = []
is_header_line = True
is_position_line = False
is_gene_line = False
gene_section_line = '],"genes":['
end_line = ']}'

In [ ]:
position_count = 0
gene_count = 0

In [ ]:


# Define a function to process a chunk of data
def process_chunk(chunk):
    for line in chunk:
        trim_line = line.strip()
        if is_header_line:
            ## only keep the "header" field content from the line
            header = trim_line[10:-14]
            is_header_line = False
            is_position_line = True
            continue
        if trim_line == gene_section_line:
            is_gene_line = True
            is_position_line = False
            continue
        elif trim_line == end_line:
            break
        else:
            if is_position_line:
                ## remove the trailing ',' if there is
                positions.append(trim_line.rstrip(','))
                position_count += 1
            if is_gene_line:
                ## remove the trailing ',' if there is
                genes.append(trim_line.rstrip(','))
                gene_count += 1

# Function to read and process the file in chunks
def process_large_file(filename, chunk_size, num_threads):
    # Create a thread-safe queue
    chunk_queue = queue.Queue()

    # Create a thread pool
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Function to read the file and put chunks into the queue
        def read_and_enqueue():
            with open(filename, 'rb') as file:
                while True:
                    chunk = file.read(chunk_size)
                    if not chunk:
                        break
                    chunk_queue.put(chunk)

        # Start the reading thread
        reading_thread = threading.Thread(target=read_and_enqueue)
        reading_thread.start()

        # Start worker threads to process the chunks
        for _ in range(num_threads):
            executor.submit(process_chunk, chunk_queue.get())

        # Wait for all threads to finish
        executor.shutdown()
        reading_thread.join()

# Usage
if __name__ == "__main__":
    filename = "large_file.txt"
    chunk_size = 1024  # Adjust the chunk size as needed
    num_threads = 4   # Adjust the number of threads as needed

    process_large_file(filename, chunk_size, num_threads)

In [ ]:
import threading
import queue
import concurrent.futures
import pandas as pd
import json
import gzip


header = ''
positions = []
genes = []
is_header_line = True
is_position_line = False
is_gene_line = False
gene_section_line = '],"genes":['
end_line = ']}'


file = "/home/marc/inters/output_test2/05_Annotation/Nirvana/snvs/Annotation.json.gz"


In [ ]:

with gzip.open(file, 'rt') as f:
    position_count = 0
    gene_count = 0
    for line in f:
        trim_line = line.strip()
        if is_header_line:
            ## only keep the "header" field content from the line
            header = trim_line[10:-14]
            is_header_line = False
            is_position_line = True
            continue
        if trim_line == gene_section_line:
            is_gene_line = True
            is_position_line = False
            continue
        elif trim_line == end_line:
            break
        else:
            if is_position_line:
                ## remove the trailing ',' if there is
                positions.append(trim_line.rstrip(','))
                position_count += 1
            if is_gene_line:
                ## remove the trailing ',' if there is
                genes.append(trim_line.rstrip(','))
                gene_count += 1

print ('number of positions:', position_count)
print ('number of genes:', gene_count)